In [2]:
# Import modules/packages
import os

# Import third-party modules/packages
import utils

In [2]:
test_file = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz"

In [3]:
log = utils.LogFile("sub-C01.log")

In [4]:
dwi_test = utils.NiiFile(test_file)

In [6]:
dwi_test.

('/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01',
 'sub-C01_ses-001_bval-b2000_run-01_dwi',
 '.nii.gz')

In [5]:
dwi_test.file = dwi_test.abs_path(); dwi_test.file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz'

In [3]:
fsl_info = utils.Command("fslinfo")

In [4]:
fsl_info.check_dependency()

True

In [8]:
fsl_info.cmd_list.append(dwi_test.file)

In [9]:
fsl_info.run(log)

Running: fslinfo /mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz
data_type	FLOAT32
dim1		96
dim2		96
dim3		84
dim4		68
datatype	16
pixdim1		1.667000
pixdim2		1.667000
pixdim3		1.700000
pixdim4		5.552192
cal_max		0.000000
cal_min		0.000000
file_type	NIFTI-1+



(0, None, None)

In [10]:
type(log)

utils.LogFile

In [13]:
os.path.splitext(dwi_test.rm_ext())

('/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi',
 '')

In [12]:
dwi_test.rm_ext()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi'